In [23]:
!pymol

QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-jovyan'
 PyMOL(TM) 2.5.2 - Incentive Product
 Copyright (C) Schrodinger, LLC
 
 This Executable Build integrates and extends Open-Source PyMOL.
No License File - For Evaluation Only (30 days remaining)
 Detected OpenGL version 3.1. Shaders available.
 Geometry shaders not available
 Detected GLSL version 1.40.
 OpenGL graphics engine:
  GL_VENDOR:   VMware, Inc.
  GL_RENDERER: llvmpipe (LLVM 10.0.0, 128 bits)
  GL_VERSION:  3.1 Mesa 20.0.8
 Detected 12 CPU cores.  Enabled multithreaded rendering.
 CmdLoad: "" loaded as "example".
PyMOL>h_add
 Save: wrote "/work/Compartido/ali.pdb".
The X11 connection broke (error 1). Did the X11 server die?


In [1]:
_aa_codes =  {
    'A' : 'ala',
    'B' : 'ace',
    'C' : 'cys',
    'D' : 'asp',
    'E' : 'glu',
    'F' : 'phe',
    'G' : 'gly',
    'H' : 'his',
    'I' : 'ile',
    'K' : 'lys',
    'L' : 'leu',
    'M' : 'met',
    'N' : 'asn',
    'P' : 'pro',
    'Q' : 'gln',
    'R' : 'arg',
    'S' : 'ser',
    'T' : 'thr',
    'V' : 'val',
    'W' : 'trp',
    'Y' : 'tyr',
    'Z' : 'nme',
    }

In [3]:
build_peptide("AA")

In [7]:
cmd.save("aa.cif")

In [8]:
from Bio.PDB import *
parser = MMCIFParser()
structure = parser.get_structure("ala", "aa.cif")

In [11]:
view = nglview.show_biopython(structure)
view.add_ball_and_stick()
view

NGLWidget()

In [ ]:
from pymol import cmd
for aa in "AA":
    cmd._alt(aa.lower())
    
cmd.h_add()
cmd.save("aa.pdb")

# Utils

In [10]:
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import math
import numpy as np
import nglview


In [ ]:
# Test installation

import openmm.testInstallation
openmm.testInstallation.main()

# Alanine-dipeptide pdb

In [13]:
!wget https://raw.githubusercontent.com/vitalreminds/serv/main/ala2.pdb

--2022-05-18 06:24:23--  https://raw.githubusercontent.com/vitalreminds/serv/main/ala2.pdb
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3630 (3.5K) [text/plain]
Saving to: ‘ala2.pdb.11’

ala2.pdb.11         100%[===================>]   3.54K  --.-KB/s    in 0.002s  

2022-05-18 06:24:24 (2.03 MB/s) - ‘ala2.pdb.11’ saved [3630/3630]



In [14]:
!wget https://raw.githubusercontent.com/molmod/openmm-tutorial-msbs/master/02_alanine_dipeptide/alanine-dipeptide.pdb

--2022-05-18 06:24:24--  https://raw.githubusercontent.com/molmod/openmm-tutorial-msbs/master/02_alanine_dipeptide/alanine-dipeptide.pdb
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1293 (1.3K) [text/plain]
Saving to: ‘alanine-dipeptide.pdb.10’

alanine-dipeptide.p 100%[===================>]   1.26K  --.-KB/s    in 0s      

2022-05-18 06:24:24 (24.0 MB/s) - ‘alanine-dipeptide.pdb.10’ saved [1293/1293]



In [15]:
!wget https://raw.githubusercontent.com/openmm/openmm/master/wrappers/python/tests/systems/alanine-dipeptide-explicit.pdb

--2022-05-18 06:24:24--  https://raw.githubusercontent.com/openmm/openmm/master/wrappers/python/tests/systems/alanine-dipeptide-explicit.pdb
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 204212 (199K) [text/plain]
Saving to: ‘alanine-dipeptide-explicit.pdb.10’

alanine-dipeptide-e 100%[===================>] 199.43K  --.-KB/s    in 0.1s    

2022-05-18 06:24:25 (1.84 MB/s) - ‘alanine-dipeptide-explicit.pdb.10’ saved [204212/204212]



# 1pef pdb

In [ ]:
!wget https://files.rcsb.org/download/1PEF.pdb

# Simulation

In [3]:
from sys import stdout

import matplotlib.pyplot as plt
import mdtraj
import nglview
import numpy as np
from openmm import *
from openmm.app import *
from openmm.unit import *

pdb = PDBFile('alanine-dipeptide.pdb')
modeller = Modeller(pdb.topology, pdb.positions)

forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')
modeller.addHydrogens(forcefield)
modeller.addSolvent(forcefield, model='tip3p', padding=1*nanometer)
print(modeller.topology)
# Write a PDB file to provide a topology of the solvated
# system to MDTraj below.
with open('init3.pdb', 'w') as outfile:
    PDBFile.writeFile(modeller.topology, modeller.positions, outfile)

# The modeller builds a periodic box with the solute and solvent molecules.
# PME is the method to compute long-range electristatic interactions in
# periodic systems.
system = forcefield.createSystem(
    modeller.topology, nonbondedMethod=PME, constraints=HBonds)
temperature = 300 * kelvin
pressure = 1 * bar
integrator = LangevinIntegrator(temperature, 1/picosecond, 2*femtoseconds)
system.addForce(MonteCarloBarostat(pressure, temperature))
simulation = Simulation(modeller.topology, system, integrator)
simulation.context.setPositions(modeller.positions)
simulation.minimizeEnergy()
simulation.reporters.append(DCDReporter('traj3.dcd', 100))
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True,
                                              temperature=True, elapsedTime=True))
simulation.reporters.append(StateDataReporter("scalars3.csv", 100, time=True,
                                              potentialEnergy=True, totalEnergy=True, temperature=True))
simulation.step(1000)

# The last line is only needed for Windows users,
# to close the DCD file before it can be opened by nglview.
del simulation

<Topology; 2 chains, 661 residues, 1996 atoms, 1337 bonds>
#"Step","Temperature (K)","Elapsed Time (s)"
1000,261.3805778508015,0.00013017654418945312


In [20]:
from Bio.PDB import *
parser = PDBParser()
structure = parser.get_structure("alanine", "ala2.pdb")

In [ ]:
traj3 = mdtraj.load('traj3.dcd', top='init3.pdb')
view = nglview.show_mdtraj(traj3)
view.clear_representations()
view.add_licorice()
view.add_unitcell()
view

# Biopython PDB - nglview

In [21]:
#from google.colab import output

#output.enable_custom_widget_manager()

import mdtraj
import nglview

view = nglview.show_biopython(structure)
view.add_ball_and_stick()
view

NGLWidget()

In [ ]:
for atom in structure.get_atoms():
    print(atom)

for residue in structure.get_residues():
    print(residue)

In [ ]:
res_list = Selection.unfold_entities(structure, "R")

In [ ]:
atoms_res1 = Selection.unfold_entities(res_list[0], "A")
atoms_res1

In [ ]:
view = nglview.show_biopython(res_list[0])
view

In [ ]:
residue = res_list[0]
o = residue["O"].get_vector()
c = residue["C"].get_vector()
ch3 = residue["CH3"].get_vector()
_3hh3 = residue["3HH3"].get_vector()

angle = calc_dihedral(o, c, ch3, _3hh3)
math.degrees(angle), o, c, ch3, _3hh3

In [ ]:
view = nglview.show_biopython(res_list[2])
view

# Dihedral angles - Biopython - Nglview

In [ ]:
from Bio.PDB import *
parser = PDBParser()
structure2 = parser.get_structure("alanine", "ala2.pdb")

In [9]:
view = nglview.show_biopython(structure2)
view.add_ball_and_stick()
view

NameError: name 'nglview' is not defined

In [ ]:
residues = Selection.unfold_entities(structure2, 'R')
residue = residues[1]

for atom in residue.get_atoms():
  print(atom)

In [ ]:
view = nglview.show_biopython(residue)
view.add_ball_and_stick()
view.add_licorice()
view

In [ ]:
residue

In [ ]:
# get atom coordinates as vectors
n = residue["N"].get_vector()
c = residue["C"].get_vector()
ca = residue["CA"].get_vector()
h = residue["H"].get_vector()
# center at origin
n = n - ca
c = c - ca
h = h - ca
# find rotation matrix that rotates n -120 degrees along the ca-c vector
rot = rotaxis(-3.14 * 45.0 / 180.0, c)
# apply rotation to ca-n vector
hb = h.right_multiply(rot) + ca
cb_at_origin = n.right_multiply(rot)
# put on top of ca atom
cb = cb_at_origin + ca

In [ ]:
atomd = residue["N"]
n, residue["N"].get_vector()

In [ ]:
cbp = Atom.Atom("NBP", cb.get_array(), 0.0, 0.1, " ", "NBP", "11", "N")
hbp = Atom.Atom("HBP", hb.get_array(), 0.0, 0.1, " ", "HBP", "12", "H")

In [ ]:
residue.add(cbp)
residue.add(hbp)

In [ ]:
residue.detach_child("N")

In [ ]:
view = nglview.show_biopython(residue)
view.add_ball_and_stick()
view.add_licorice()
view

# Calculate all phi psi, tau

In [ ]:
parser = PDBParser()
structure = parser.get_structure("alanine", "ala2.pdb")
structure2 = parser.get_structure("1ef", "1PEF.pdb")

# Using C-N
#ppb = PPBuilder()

# Using CA-CA
ppb = CaPPBuilder()

polypeptides = ppb.build_peptides(structure, aa_only=False)
polypeptides

In [ ]:
pp = polypeptides[0]
type(pp)

In [ ]:
pp.get_phi_psi_list()

In [ ]:
pp.get_theta_list()

In [ ]:
pp.get_ca_list()

In [ ]:
pp.get_tau_list()

# Internal_cords_module

In [ ]:
from Bio.PDB import *
parser = PDBParser()
structure = parser.get_structure("alanine", "ala2.pdb")
pymstructure = parser.get_structure("pymalanine", "aa.pdb")

In [ ]:
view = nglview.show_biopython(structure)
view.add_ball_and_stick()
view.add_licorice()
view

In [ ]:
view = nglview.show_biopython(pymstructure)
view.add_ball_and_stick()
view.add_licorice()
view

In [ ]:
for atom in structure.get_atoms():
  print(atom)

In [ ]:
models = Selection.unfold_entities(structure, "M")
model = models[0]

model.atom_to_internal_coordinates()
for r in model.get_residues():
  if r.internal_coord:
    print(
        r,
        r.internal_coord,
        r.internal_coord.get_angle("psi"),
        r.internal_coord.get_angle("phi"),
        r.internal_coord.get_angle("omega"), # or "omg"
        r.internal_coord.get_angle("chi2"),
        r.internal_coord.get_angle("CB:CA:C"),
        (r.internal_coord.get_length("-1C:0N") # i-1 to i peptide bond
        if r.internal_coord.rprev else "None")
        )


In [ ]:
models = Selection.unfold_entities(pymstructure, "M")
model = models[0]

model.atom_to_internal_coordinates()
for r in model.get_residues():
  if r.internal_coord:
    print(
        r,
        r.internal_coord,
        r.internal_coord.get_angle("psi"),
        r.internal_coord.get_angle("phi"),
        r.internal_coord.get_angle("omega"), # or "omg"
        r.internal_coord.get_angle("chi2"),
        r.internal_coord.get_angle("CB:CA:C"),
        (r.internal_coord.get_length("-1C:0N") # i-1 to i peptide bond
        if r.internal_coord.rprev else "None")
        )


In [ ]:
residues = Selection.unfold_entities(pymstructure, "R")
residue = residues[0]

ω (omega) is the angle in the chain Cα − C' − N − Cα,

φ (phi) is the angle in the chain C' − N − Cα − C'

ψ (psi) is the angle in the chain N − Cα − C' − N 

In [ ]:
psi = residue.internal_coord.pick_angle("psi")
psi

In [ ]:
residue.internal_coord.set_angle("psi", 147.9183256188407)

In [ ]:
residue1 = residues[1]

In [ ]:
phi = residue1.internal_coord.pick_angle("phi")
phi

In [ ]:
residue1.internal_coord.set_angle("phi", -146.94391460728016)

In [ ]:
model.internal_to_atom_coordinates()

In [ ]:
view = nglview.show_biopython(structure)
view.add_ball_and_stick()
view.add_licorice()
view

In [ ]:
view = nglview.show_biopython(model)
view.add_ball_and_stick()
view.add_licorice()
view

In [ ]:
models = Selection.unfold_entities(pymstructure, "M")
model = models[0]

model.atom_to_internal_coordinates()
for r in model.get_residues():
  if r.internal_coord:
    print(
        r,
        r.internal_coord,
        r.internal_coord.get_angle("psi"),
        r.internal_coord.get_angle("phi"),
        r.internal_coord.get_angle("omega"), # or "omg"
        r.internal_coord.get_angle("chi2"),
        r.internal_coord.get_angle("CB:CA:C"),
        (r.internal_coord.get_length("-1C:0N") # i-1 to i peptide bond
        if r.internal_coord.rprev else "None")
        )

In [2]:

import re
import pymol
cmd = __import__("sys").modules["pymol.cmd"]
from pymol import setting
from pymol import parsing
import threading

from pymol.cmd import DEFAULT_ERROR, DEFAULT_SUCCESS, _raising, \
     is_list,  is_string, is_error

QuietException = parsing.QuietException

_prefix = "_tmp_editor"
tmp_wild = _prefix + "*"
tmp_editor = _prefix + "0"
tmp_connect = _prefix + "_con"
tmp_domain = _prefix + "_dom"
tmp1 = _prefix + "1"
tmp2 = _prefix + "2"
tmp3 = _prefix + "3"
tmp4 = _prefix + "4"

def attach_amino_acid(selection,amino_acid,center=0,animate=-1,object="",hydro=1,ss=-1,_self=cmd):
    '''
ARGUMENTS
    selection = str: named selection of single N or C atom
    amino_acid = str: fragment name to load from fragment library
    center = bool: center on new terminus (pk1)
    animate = int: animate centering
    object = str: name of new object (if selection is none)
    hydro = int (-1/0/1): keep hydrogens
    ss = int: Secondary structure 1=alpha helix, 2=antiparallel beta, 3=parallel beta, 4=flat
    '''
    r = DEFAULT_SUCCESS
    ss = int(ss)
    center = int(center)

    if hydro<0:
        hydro = not int(_self.get_setting_boolean("auto_remove_hydrogens"))
    if (selection not in _self.get_names('all')
            if selection == 'pk1' # legacy, calling functions should pass '?pk1'
            else _self.count_atoms(selection) == 0):
        if object == "":
            object = amino_acid
        # create new object
        if amino_acid in _self.get_names("objects"):
            print("Error: an object with than name already exists")
            raise QuietException
        r = _self.fragment(amino_acid,object)
        if not hydro:
            _self.remove("(hydro and %s)"%object)
        if _self.count_atoms("((%s) and name C)"%object):
            _self.edit("((%s) and name C)"%object)
        elif _self.count_atoms("((%s) and name N)"%object):
            _self.edit("((%s) and name N)"%object)
    elif _self.select(tmp_connect,"(%s) & elem N,C"%selection) != 1:
        print("Error: invalid connection point: must be one atom, name N or C.")
        _self.delete(tmp_wild)
        raise QuietException
    elif amino_acid in ["nhh","nme"] and _self.select(tmp_connect,"(%s) & elem C"%selection) != 1:
        print("Error: invalid connection point: must be C for residue '%s'"%(amino_acid))
        _self.delete(tmp_wild)
        raise QuietException
    elif amino_acid in ["ace"] and _self.select(tmp_connect,"(%s) & elem N"%selection) != 1:
        print("Error: invalid connection point: must be N for residue '%s'"%(amino_acid))
        _self.delete(tmp_wild)
        raise QuietException
    else:
        if ss<0:
            ss = _self.get_setting_int("secondary_structure")
        if ss:
            if ss==1: # helix
                phi=-57.0
                psi=-47.0
            elif ss==2: # antipara-beta
                phi=-139.0
                psi=135.0
            elif ss==3: # para-beta
                phi=-119.0
                psi=113.0
            else:
                phi=180.0
                psi=180.0
        _self.fragment(amino_acid,tmp_editor, origin=0)
        if _self.count_atoms("elem N",domain=tmp_connect):
            tmp = [ None ]
            _self.iterate(tmp_connect,"tmp[0]=resv", space={ 'tmp' : tmp })
            tmp[0] = str(tmp[0]-1) # counting down
            _self.alter(tmp_editor,"resi=tmp[0]",space={ 'tmp' : tmp})
            _self.set_geometry(tmp_connect, 3, 3) # make nitrogen planar
            _self.fuse("(%s and name C)"%(tmp_editor),tmp_connect,2)
            _self.select(tmp_domain, "byresi (pk1 | pk2)")

            if not hydro:
                _self.remove("(pkmol and hydro)")

            if ((_self.select(tmp1,"?pk1",domain=tmp_domain)==1) and
                (_self.select(tmp2,"?pk2",domain=tmp_domain)==1)):

                if ((_self.select(tmp3,"(name CA,CH3 & nbr. ?pk1)",domain=tmp_domain)==1) and
                    (_self.select(tmp4,"(name CA,CH3 & nbr. ?pk2)",domain=tmp_domain)==1)):
                    _self.set_dihedral(tmp4,tmp2,tmp1,tmp3,180.0)

                if hydro:
                    _self.h_fix(tmp2) # fix hydrogen position

                if ss:
                    if amino_acid[0:3]!='pro':
                        if ((_self.select(tmp4,
                                          "(!(resn PRO) & name C  & nbr. (name CA & nbr. "+tmp2+"))",
                                          domain=tmp_domain)==1) and
                            (_self.select(tmp3,
                                          "(!(resn PRO) & name CA & nbr. "+tmp2+")",
                                          domain=tmp_domain)==1)):
                            _self.set_dihedral( # PHI
                                tmp4, # C
                                tmp3, # CA
                                tmp2, # N
                                tmp1, # C
                                phi)

                    if ((_self.select(tmp4,"(name N & nbr. (name CA & nbr. "+tmp1+"))",
                                      domain=tmp_domain)==1) and
                        (_self.select(tmp3,"(name CA & nbr. "+tmp1+")",domain=tmp_domain)==1)):
                        _self.set_dihedral( # PSI (n-1)
                            tmp2, # N
                            tmp1, # C
                            tmp3, # CA
                            tmp4, # N
                            psi)

            sele = ("(name N & (byres nbr. %s) &! (byres %s))"% (tmp_connect,tmp_connect))
            if _self.select(tmp1,sele,domain=tmp_domain):
                _self.edit(tmp1)
                if center:
                    _self.center(tmp1,animate=animate)
        elif _self.count_atoms("elem C",domain=tmp_connect): # forward
            tmp = [ None ]
            _self.iterate(tmp_connect,"tmp[0]=resv", space={ 'tmp' : tmp })
            tmp[0] = str(tmp[0]+1) # counting up
            _self.alter(tmp_editor,"resi=tmp[0]",space={ 'tmp' : tmp})
            _self.set_geometry(tmp_editor + " & name N", 3, 3) # make nitrogen planar
            _self.fuse("(%s and name N)"%tmp_editor,tmp_connect,2)
            _self.select(tmp_domain, "byresi (pk1 | pk2)")

            if not hydro:
                _self.remove("(pkmol and hydro)")

            if (( _self.select(tmp1,"?pk1",domain=tmp_domain)==1) and
                ( _self.select(tmp2,"?pk2",domain=tmp_domain)==1)):

                if ((_self.select(tmp3,"(name CA,CH3 & nbr. ?pk1)",domain=tmp_domain)==1) and
                    (_self.select(tmp4,"(name CA,CH3 & nbr. ?pk2)",domain=tmp_domain)==1)):
                    _self.set_dihedral(tmp4,tmp2,tmp1,tmp3,180.0)
                if hydro:
                    _self.h_fix("pk1") # fix hydrogen position
                if ss:
                    if hydro and amino_acid[0:3]=='nhh': # fix amide hydrogens
                        if ((_self.select(tmp3,"(name H1 & nbr. "+tmp1+")",domain=tmp_domain)==1) and
                            (_self.select(tmp4,"(name O & nbr. "+tmp2+")",domain=tmp_domain)==1)):
                            _self.set_dihedral(
                                tmp4, # O
                                tmp2, # C
                                tmp1, # N
                                tmp3, # H1
                                180)
                    if amino_acid[0:3]!='pro':
                        if ((_self.select(tmp3,"(name CA & nbr. "+tmp1+")",domain=tmp_domain)==1) and
                            (_self.select(tmp4,"(name C & nbr. (name CA & nbr. "+tmp1+"))",domain=tmp_domain)==1)):
                            _self.set_dihedral( # PHI
                                tmp2, # C
                                tmp1, # N
                                tmp3, # CA
                                tmp4, # C
                                phi)
                    if ((_self.select(tmp3,"(name CA & nbr. "+tmp2+")",domain=tmp_domain)==1) and
                        (_self.select(tmp4,"(name N & nbr. (name CA & nbr. "+tmp2+"))",domain=tmp_domain)==1)):
                        _self.set_dihedral( # PSI (n-1)
                            tmp4, # N
                            tmp3, # CA
                            tmp2, # C
                            tmp1, # N
                            psi)
            sele = ("(name C & (byres nbr. %s) & !(byres %s))"% (tmp_connect,tmp_connect))
            if _self.select(tmp1,sele,domain=tmp_domain):
                _self.edit(tmp1)
                if center:
                    _self.center(tmp1,animate=animate)
            else:
                _self.unpick()
        elif _self.count_atoms("((%s) and elem H)"%selection):
            print("Error: please pick a nitrogen or carbonyl carbon to grow from.")
            _self.delete(tmp_wild)
            raise QuietException
        else:
            print("Error: unable to attach fragment.")
            _self.delete(tmp_wild)
            raise QuietException
    _self.delete(tmp_wild)

    return r



def build_peptide(sequence,_self=cmd): # legacy
    for aa in sequence:
        attach_amino_acid("pk1",_aa_codes[aa])

# 1PEF

In [ ]:
parser = PDBParser()
structure = parser.get_structure("1pef", "1PEF.pdb")

In [ ]:
models = Selection.unfold_entities(structure, "M")
model = models[0]

model.atom_to_internal_coordinates()
for r in model.get_residues():
  if r.internal_coord:
    print(
        r,
        r.internal_coord.get_angle("psi"),
        r.internal_coord.get_angle("phi"),
        r.internal_coord.get_angle("omega"), # or "omg"
        r.internal_coord.get_angle("chi2"),
        )

In [ ]:
view = nglview.show_biopython(structure)
view.add_ball_and_stick()
view.add_licorice()
view

In [ ]:
residues = Selection.unfold_entities(structure, "R")
residue = residues[1]

In [ ]:
view = nglview.show_biopython(residue)
view.add_ball_and_stick()
view.add_licorice()
view

In [ ]:
chi2 = residue.internal_coord.pick_angle("chi2")
chi2

In [ ]:
residue.internal_coord.set_angle("chi2", 130.0)

In [ ]:
model.internal_to_atom_coordinates()

In [ ]:
view = nglview.show_biopython(model)
view.add_ball_and_stick()
view.add_licorice()
view